# Langchain Testing

In [26]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# make it easy to switch
from langchain.llms import Ollama
from langchain.llms import OpenAI
from langchain.chains import SimpleSequentialChain

In [25]:
model_name = "llama3.1"

# this is for Ollama
# Initialize the LLM connections
llm_1 = Ollama(model=model_name)
llm_2 = Ollama(model=model_name)
# OpenAI

'''
llm_2 = OpenAI(
    base_url="<path>",
    model=model_name,
    api_key="NONE", 
    max_tokens=1000,
    temperature=0.7,
    top_p=0.9 
)
'''

'\nllm_2 = OpenAI(\n    base_url="<path>",\n    model=model_name,\n    api_key="NONE", \n    max_tokens=1000,\n    temperature=0.7,\n    top_p=0.9 \n)\n'

## ChatGPT example

In [13]:
# Define a prompt template for security vulnerability analysis
prompt_template = PromptTemplate(
    input_variables=["vulnerability_desc", "severity"],
    template="""
    You are a cybersecurity expert analyzing vulnerabilities.
    
    Vulnerability: {vulnerability_desc}
    Severity Rating: {severity}
    
    - Briefly summarize the vulnerability.
    - Validate if the severity rating seems accurate.
    - Provide recommendations to mitigate this vulnerability.
    
    Provide a structured response.
    """
)

# Example vulnerability input
example_vulnerability = {
    "vulnerability_desc": "SQL injection allows attackers to execute arbitrary SQL commands on a database.",
    "severity": "High"
}

# Format the prompt with real data
formatted_prompt = prompt_template.format(
    vulnerability_desc=example_vulnerability["vulnerability_desc"],
    severity=example_vulnerability["severity"]
)

# Run the model and get the response
response = llm.invoke(formatted_prompt)

# Print the LLM's analysis
print(response)

**Vulnerability Summary**

The identified vulnerability is an SQL injection (SQLi) attack, which allows attackers to execute arbitrary SQL commands on a database. This occurs when user-input data is not properly sanitized or validated, enabling malicious users to inject malicious SQL code that can manipulate or extract sensitive data.

**Severity Rating Validation**

The severity rating of "High" seems accurate for this vulnerability. SQL injection attacks can lead to significant consequences, including:

1. **Data breaches**: Attackers may gain unauthorized access to sensitive data, such as financial information, personal identifiable information (PII), or intellectual property.
2. **System compromise**: Successful exploitation can allow attackers to modify database schema, delete or alter critical data, or even execute malicious code on the underlying operating system.
3. **Financial impact**: Depending on the nature of the compromised data and systems, SQLi attacks can result in sig

### Add Risk Scoring

In [35]:

risk_scoring_prompt = PromptTemplate(
    input_variables=["analysis"],
    template="""
    You are a cybersecurity risk assessor.

    Based on the following vulnerability analysis:
    {analysis}

    - Assign a risk score from 1 to 10 (1 = minimal risk, 10 = critical risk).
    
    Respond only with: "Risk Score: X" where X is the number.
    """
)

# Invoke the model to get a risk score from the response generated above
risk_score_response = llm_1.invoke(risk_scoring_prompt.format(analysis=response))

# Print the risk score
print(risk_score_response)


Risk Score: 3


### Justify the Risk Score

In [36]:
risk_justification_prompt = PromptTemplate(
    input_variables=["analysis", "risk_score"],
    template="""
    You are a cybersecurity expert.

    Based on the following vulnerability analysis:
    {analysis}

    The assigned risk score is: {risk_score}

    - Justify why this risk score is appropriate based on the severity, exploitability, and potential impact.
    
    Provide a concise and structured explanation.
    """
)

# Invoke the model to get justification for the risk score
risk_justification_response = llm.invoke(
    risk_justification_prompt.format(analysis=response, risk_score=risk_score_response)
)

# Print the justification
print(risk_justification_response)


**Risk Assessment**

Based on the provided vulnerability analysis, I justify that the assigned risk score of **3** is appropriate. Here's a concise and structured explanation:

**Severity (1)**: The presence of a known exploited vulnerability, **CVE-2024-55580**, indicates a high severity level (scored 1) as it has already been successfully exploited in the wild.

**Exploitability (1)**: Given that **CVE-2024-55580** is on the KEV list, it's considered highly exploitable (scored 1). The ease of exploitation increases the risk score.

**Potential Impact (1)**: With a potential Denial of Service (DoS) and Unvalidated input or data exposure related to CWEs **22** and **352**, there is a moderate to high potential impact (scored 1), assuming successful exploitation. This could lead to significant downtime, data breaches, or other security incidents.

Considering these factors, the risk score of **3** is justified as it reflects the combination of severity, exploitability, and potential imp

### Provide Best Security Practices

In [38]:
# Step 3: Provide Best Security Practices
best_practices_prompt = PromptTemplate(
    input_variables=["analysis", "risk_score", "justification"],
    template="""
    You are a cybersecurity advisor.

    Based on the following information:
    Vulnerability Analysis: {analysis}
    Risk Score: {risk_score}
    Justification: {justification}

    - List the top 3 best security practices to mitigate this vulnerability.
    - Ensure the recommendations are practical and aligned with industry standards.
    """
)

# Invoke the model to get best security practices
best_practices_response = llm.invoke(
    best_practices_prompt.format(
        analysis=response,
        risk_score=risk_score_response,
        justification=risk_justification_response
    )
)

# Print the best security practices
print(best_practices_response)


Based on the provided vulnerability analysis, I recommend implementing the following top 3 best security practices to mitigate **CVE-2024-55580**:

1.  **Patch Management**: Immediately apply the available patch for **CVE-2024-55580** to ensure that the known exploited vulnerability is addressed. This can be done by applying the latest updates, patches, or hotfixes from the affected vendor.

    *   Recommendation: Regularly review and update your systems with the latest security patches.
2.  **Input Validation**: Implement input validation mechanisms to prevent Unvalidated input or data exposure related to CWEs **22** and **352**. This can be achieved by:
    *   Validating user inputs through robust checks, such as whitelisting, sanitization, and parameterized queries.
    *   Regularly reviewing and updating your application's input validation mechanisms.

3.  **Secure Configuration**: Ensure that your systems are configured securely to prevent potential Denial of Service (DoS) atta

### All Together

In [39]:
# Step 1: Define the LLM connection (use 'llm_1')

# ================================
# Step 2: Define all Prompt Templates
# ================================

# Template 1: Initial Vulnerability Analysis
vulnerability_analysis_prompt = PromptTemplate(
    input_variables=["vulnerability_desc", "severity"],
    template="""
    You are a cybersecurity expert analyzing vulnerabilities.

    Vulnerability: {vulnerability_desc}
    Severity Rating: {severity}

    - Briefly summarize the vulnerability.
    - Validate if the severity rating seems accurate.
    - Provide recommendations to mitigate this vulnerability.

    Provide a structured response.
    """
)

# Template 2: Risk Scoring
risk_scoring_prompt = PromptTemplate(
    input_variables=["analysis"],
    template="""
    You are a cybersecurity risk assessor.

    Based on the following vulnerability analysis:
    {analysis}

    - Assign a risk score from 1 to 10 (1 = minimal risk, 10 = critical risk).
    
    Respond only with: "Risk Score: X" where X is the number.
    """
)

# Template 3: Justify the Risk Score
risk_justification_prompt = PromptTemplate(
    input_variables=["analysis", "risk_score"],
    template="""
    You are a cybersecurity expert.

    Based on the following vulnerability analysis:
    {analysis}

    The assigned risk score is: {risk_score}

    - Justify why this risk score is appropriate based on the severity, exploitability, and potential impact.
    
    Provide a concise and structured explanation.
    """
)

# Template 4: Best Security Practices
best_practices_prompt = PromptTemplate(
    input_variables=["analysis", "risk_score", "justification"],
    template="""
    You are a cybersecurity advisor.

    Based on the following information:
    Vulnerability Analysis: {analysis}
    Risk Score: {risk_score}
    Justification: {justification}

    - List the top 3 best security practices to mitigate this vulnerability.
    - Ensure the recommendations are practical and aligned with industry standards.
    """
)

# ================================
# Step 3: Define the Flow
# ================================

# Example vulnerability input
vulnerability_input = {
    "vulnerability_desc": "SQL injection allows attackers to execute arbitrary SQL commands on a database.",
    "severity": "High"
}

# Step 3.1: Perform initial vulnerability analysis
print("Starting initial vulnerability analysis...")
formatted_prompt_1 = vulnerability_analysis_prompt.format(
    vulnerability_desc=vulnerability_input["vulnerability_desc"],
    severity=vulnerability_input["severity"]
)
response_analysis = llm_1.invoke(formatted_prompt_1)
print("Vulnerability analysis completed.\n")
print(f"Analysis Response:\n{response_analysis}\n")

# Step 3.2: Perform risk scoring
print("Assigning a risk score...")
formatted_prompt_2 = risk_scoring_prompt.format(analysis=response_analysis)
response_risk_score = llm_1.invoke(formatted_prompt_2)
print("Risk score assigned.\n")
print(f"Risk Score Response:\n{response_risk_score}\n")

# Step 3.3: Justify the risk score
print("Justifying the risk score...")
formatted_prompt_3 = risk_justification_prompt.format(
    analysis=response_analysis, risk_score=response_risk_score
)
response_justification = llm_1.invoke(formatted_prompt_3)
print("Risk score justification completed.\n")
print(f"Justification Response:\n{response_justification}\n")

# Step 3.4: Provide best security practices
print("Recommending best security practices...")
formatted_prompt_4 = best_practices_prompt.format(
    analysis=response_analysis, risk_score=response_risk_score, justification=response_justification
)
response_practices = llm_1.invoke(formatted_prompt_4)
print("Best security practices generated.\n")
print(f"Best Practices Response:\n{response_practices}\n")

# ================================
# Step 4: Final Output
# ================================

print("All steps completed successfully! Here is the consolidated output:\n")
print("1. Vulnerability Analysis:")
print(response_analysis)
print("\n2. Risk Score:")
print(response_risk_score)
print("\n3. Justification for Risk Score:")
print(response_justification)
print("\n4. Best Security Practices:")
print(response_practices)


Starting initial vulnerability analysis...
Vulnerability analysis completed.

Analysis Response:
**Vulnerability Summary**

The SQL injection vulnerability allows attackers to execute arbitrary SQL commands on a database, enabling them to:

* Extract sensitive data
* Modify or delete data
* Execute administrative commands (e.g., dropping tables or databases)
* Bypass authentication and authorization controls

This vulnerability occurs when an application concatenates user input into SQL queries without proper sanitization, allowing attackers to inject malicious SQL code.

**Severity Rating Validation**

The severity rating of High is accurate due to the following reasons:

1. **Potential impact**: SQL injection can lead to unauthorized data access, modification, or deletion, which can have severe consequences for organizations.
2. **Ease of exploitation**: An attacker with basic knowledge of SQL and web application vulnerabilities can exploit this weakness using various tools and techn

## broke??

In [20]:
# chained input
prompt_template = PromptTemplate(
    input_variables=["context", "question", "vulnerabilities"],
    template="""
You are a seasoned security analyst and develop who specializes in API programming and testing.
Based on the following context, prioritize vulnerabilities, considering potential for rapid exploitation and impact severity.
 
Context:
{context}
 
Question:
{question}
 
Vulnerabilities:
{vulnerabilities}
 
Answer:
"""
)

inputs = {
    "context": " First Vulnerabilities are evaluated based on the Known Exploited Vulnerabilities (KEV) list and Secondly CWE 79,CWE 787,CWE 22,CWE 352,CWE 89,CWE 125,CWE 78,CWE 416,CWE 862,CWE 434 priorities.",
    "question": "Rank vulnerabilities based on the given context?",
    "vulnerabilities": "CVE-2023-24955, CVE-2024-55580, CVE-2023-44981,CVE-2024-1597,CVE-2023-7272,CVE-2016-1000027,CVE-2023-1000028,CVE-2023-1000029,CVE-2023-1000030,CVE-2023-1000031"
}

## Single chain

In [31]:
vuln1_chain = LLMChain(
    llm=llm_1,
    prompt=prompt_template
)
 
# Execute the query for vuln1 chain 
try:
    response_1 = vuln1_chain.invoke(inputs)
except ValueError as ve:
    print(f"ValueError: {ve}")
except Exception as e:
    raise RuntimeError(f"Failed to execute query: {e}")

print(response_1)

{'context': ' First Vulnerabilities are evaluated based on the Known Exploited Vulnerabilities (KEV) list and Secondly CWE 79,CWE 787,CWE 22,CWE 352,CWE 89,CWE 125,CWE 78,CWE 416,CWE 862,CWE 434 priorities.', 'question': 'Rank vulnerabilities based on the given context?', 'vulnerabilities': 'CVE-2023-24955, CVE-2024-55580, CVE-2023-44981,CVE-2024-1597,CVE-2023-7272,CVE-2016-1000027,CVE-2023-1000028,CVE-2023-1000029,CVE-2023-1000030,CVE-2023-1000031', 'text': 'A fun challenge!\n\nTo prioritize the vulnerabilities, I\'ll follow the provided context:\n\n1. Evaluate based on the Known Exploited Vulnerabilities (KEV) list.\n2. Consider CWEs in descending order of priority: CWE 22, CWE 352, CWE 78, CWE 416, CWE 862, CWE 434, CWE 125, CWE 89, CWE 787, CWE 79.\n\nAfter researching each CVE and correlating it with the CWE list, I\'ll provide a ranked list. Keep in mind that this is a manual process, and there might be cases where additional research is required to accurately assign a priority.\

### Simple Sequential Chain

In [41]:
'''
prompt1 = PromptTemplate(
    input_variables=["input_text"],
    template="Analyze the following text output and summarize it: {input_text}"
)
  
prompt2 = PromptTemplate(
    input_variables=["summary"],
    template="Based on the summary: {summary}, provide CVSS v4.0 Ratings each vulnerability from NVD Qualitative Severity Ratings."
)

# response from
input_text = response_1

chain1 = LLMChain(llm=llm_1, prompt=prompt1)
 
chain2 = LLMChain(llm=llm_2, prompt=prompt2)
 
sequential_chain = SimpleSequentialChain(chains=[chain1, chain2])
 
initial_input = response
 
output = sequential_chain.run(initial_input)
print("Response:", response)
print("Recommendations :", output)
'''

'\nprompt1 = PromptTemplate(\n    input_variables=["input_text"],\n    template="Analyze the following text output and summarize it: {input_text}"\n)\n  \nprompt2 = PromptTemplate(\n    input_variables=["summary"],\n    template="Based on the summary: {summary}, provide CVSS v4.0 Ratings each vulnerability from NVD Qualitative Severity Ratings."\n)\n\n# response from\ninput_text = response_1\n\nchain1 = LLMChain(llm=llm_1, prompt=prompt1)\n \nchain2 = LLMChain(llm=llm_2, prompt=prompt2)\n \nsequential_chain = SimpleSequentialChain(chains=[chain1, chain2])\n \ninitial_input = response\n \noutput = sequential_chain.run(initial_input)\nprint("Response:", response)\nprint("Recommendations :", output)\n'